In [2]:
import pandas as pd
import graphlab
%matplotlib inline

This non-commercial license of GraphLab Create for academic use is assigned to mfrancod@uci.edu and will expire on July 23, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1525328789.log


In [19]:
if __name__ == '__main__':
    dfusers = pd.read_csv('users_list_en.csv')
    dfusers = pd.read_csv('users_list_en.csv')
    dfusers = dfusers[['SEX_ID', 'AGE','USER_ID_hash']]
    
    dfcoupvisit = pd.read_csv('coupon_visit_train_en.csv')
    dfcoupvisit = dfcoupvisit[['PURCHASE_FLAG','COUPON_ID_hash']].drop_duplicates()
    
    dfcoupdetail = pd.read_csv('coupon_detail_train_en.csv')
    dfcoupdetail = dfcoupdetail[['I_DATE','USER_ID_hash', 'COUPON_ID_hash']]
    
    dfcouplist = pd.read_csv('coupon_list_train_en.csv')
    dfcouplist = dfcouplist[['CAPSULE_TEXT_en','PRICE_RATE','DISCOUNT_PRICE','DISPPERIOD','VALIDPERIOD','USABLE_DATE_MO','USABLE_DATE_TUE','USABLE_DATE_WED','USABLE_DATE_THU','USABLE_DATE_FRI','USABLE_DATE_SAT','USABLE_DATE_SUN','USABLE_DATE_HOLIDAY','USABLE_DATE_BEFORE_HOLIDAY','large_area_name_en','ken_name_en','small_area_name_en','COUPON_ID_hash']]

    dfcouplist['PRICE']=100*dfcouplist['DISCOUNT_PRICE']/dfcouplist['PRICE_RATE']
    del dfcouplist['DISCOUNT_PRICE']
    del dfcouplist['PRICE_RATE']    
    dfcouplist['TIME']=(dfcouplist['USABLE_DATE_MO']+dfcouplist['USABLE_DATE_TUE']+dfcouplist['USABLE_DATE_WED']
                        +dfcouplist['USABLE_DATE_THU']+dfcouplist['USABLE_DATE_FRI']+dfcouplist['USABLE_DATE_SAT']
                        +dfcouplist['USABLE_DATE_SUN']+dfcouplist['USABLE_DATE_HOLIDAY']+dfcouplist['USABLE_DATE_BEFORE_HOLIDAY'])/9
    del dfcouplist['USABLE_DATE_MO']    
    del dfcouplist['USABLE_DATE_TUE']    
    del dfcouplist['USABLE_DATE_WED']    
    del dfcouplist['USABLE_DATE_THU']    
    del dfcouplist['USABLE_DATE_FRI']    
    del dfcouplist['USABLE_DATE_SAT']    
    del dfcouplist['USABLE_DATE_SUN']    
    del dfcouplist['USABLE_DATE_HOLIDAY']    
    del dfcouplist['USABLE_DATE_BEFORE_HOLIDAY']    
    
    
    
    df1 = pd.merge(dfcoupdetail, dfcouplist,
                       left_on='COUPON_ID_hash', right_on='COUPON_ID_hash')
    df2 = pd.merge(df1, dfcoupvisit,on='COUPON_ID_hash')

    df = pd.merge(df2, dfusers, on='USER_ID_hash')

    df['I_DATE'] = df['I_DATE'].fillna('N/A').astype(str)
    df['USER_ID_hash'] = df['USER_ID_hash'].fillna('N/A').astype(str)
    df['COUPON_ID_hash'] = df['COUPON_ID_hash'].fillna('N/A').astype(str)
    
    df['CAPSULE_TEXT_en'] = df['CAPSULE_TEXT_en'].fillna('N/A').astype(str)

    df['DISPPERIOD'] = df['DISPPERIOD'].fillna(0).astype(int)
    df['VALIDPERIOD'] = df['VALIDPERIOD'].fillna(0).astype(int)
    df['PRICE'] = df['PRICE'].fillna(0).astype(int)
    df['TIME'] = df['TIME'].fillna(0).astype(int)
    
    df['large_area_name_en'] = df['large_area_name_en'].fillna('N/A').astype(str)
    df['ken_name_en'] = df['ken_name_en'].fillna('N/A').astype(str)
    df['small_area_name_en'] = df['small_area_name_en'].fillna('N/A').astype(str)
    
    df['SEX_ID'] = df['SEX_ID'].fillna('N/A').astype(str)
    df['AGE'] = df['AGE'].fillna(0).astype(int)

    dfbydate=df.sort_values(by=['I_DATE'])


    (n,m)=dfbydate.shape
    print('dimensions of df:',n,m)
    fortrain=int(n*8/10)
    fortest=n-fortrain
    print('for training we use :',fortrain)
    print('for test: ',fortest)

    train=df.iloc[0:fortrain]
    test=df.iloc[fortrain:n]
    print(train.shape,test.shape)


    train_data = graphlab.SFrame(train)
    test_data = graphlab.SFrame(test)

    print(train_data.head())
    item_sim_model_pearson = graphlab.item_similarity_recommender.create(train_data, user_id='USER_ID_hash', item_id='COUPON_ID_hash', similarity_type='pearson')
    item_sim_model_cosine = graphlab.item_similarity_recommender.create(train_data, user_id='USER_ID_hash', item_id='COUPON_ID_hash', similarity_type='cosine')
    item_sim_model_jaccard = graphlab.item_similarity_recommender.create(train_data, user_id='USER_ID_hash', item_id='COUPON_ID_hash', similarity_type='jaccard')

    model_performance = graphlab.compare(test_data, [item_sim_model_cosine, item_sim_model_pearson,item_sim_model_jaccard])
    graphlab.show_comparison(model_performance,[item_sim_model_cosine, item_sim_model_pearson,item_sim_model_jaccard])

('dimensions of df:', 334216, 14)
('for training we use :', 267372)
('for test: ', 66844)
((267372, 14), (66844, 14))
+---------------------+-------------------------------+
|        I_DATE       |          USER_ID_hash         |
+---------------------+-------------------------------+
| 2012-03-28 15:06:06 | d9dca3cb44bab12ba313eaa681... |
| 2012-03-28 15:06:06 | d9dca3cb44bab12ba313eaa681... |
| 2012-03-26 23:26:24 | dd8a79645e8170db6a91bad1d8... |
| 2012-03-26 23:26:24 | dd8a79645e8170db6a91bad1d8... |
| 2011-11-12 15:09:53 | dd8a79645e8170db6a91bad1d8... |
| 2011-11-12 15:09:53 | dd8a79645e8170db6a91bad1d8... |
| 2012-04-09 22:49:16 | dd8a79645e8170db6a91bad1d8... |
| 2012-04-09 22:49:16 | dd8a79645e8170db6a91bad1d8... |
| 2012-05-16 21:18:39 | dd8a79645e8170db6a91bad1d8... |
| 2012-05-16 21:18:39 | dd8a79645e8170db6a91bad1d8... |
+---------------------+-------------------------------+
+-------------------------------+------------------+------------+-------------+
|         COUPON_I

Recsys training: model = item_similarity

Warning: Ignoring columns I_DATE, CAPSULE_TEXT_en, DISPPERIOD, VALIDPERIOD, large_area_name_en, ken_name_en, small_area_name_en, PRICE, TIME, PURCHASE_FLAG, SEX_ID, AGE;

To use one of these as a target column, set target = <column_name>

and use a method that allows the use of a target.

...

recommendations finished on 7000/10499 queries. users per second: 20272.1

recommendations finished on 8000/10499 queries. users per second: 20026.5

recommendations finished on 9000/10499 queries. users per second: 19867.8

recommendations finished on 10000/10499 queries. users per second: 18751.5


Precision and recall summary statistics by cutoff
+--------+------------------+-------------------+
| cutoff |  mean_precision  |    mean_recall    |
+--------+------------------+-------------------+
|   1    | 0.0089532336413  | 0.000677953507375 |
|   2    | 0.00728640822936 |  0.00121781695951 |
|   3    | 0.00625456392672 |  0.0014456285204  |
|   4    | 0.0060958186494  |  0.00179500605296 |
|   5    | 0.00577197828365 |  0.00210799559901 |
|   6    | 0.00541321395688 |  0.0023353845203  |
|   7    | 0.00564679629352 |  0.00289181812678 |
|   8    | 0.00551242975521 |  0.0032932032674  |
|   9    | 0.00536559037369 |  0.00366385824889 |
|   10   | 0.00513382226879 |  0.0038966591269  |
+--------+------------------+-------------------+
[10 rows x 3 columns]

Model compare metric: precision_recall
Canvas is updated and available in a tab in the default browser.
